CODIGO REFATORADO - ML_STATE_OF_DATA_25.ipynb


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline   # <<< usar o pipeline do imblearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import joblib


def carregar_dados(caminho_csv):
    df = pd.read_csv(caminho_csv).drop_duplicates()

    colunas = {
    # Identidade
    "0.a_token": "id",
    "1.a_idade": "idade",
    "1.b_genero": "genero",
    "1.c_cor/raca/etnia": "cor",
    "1.d_pcd": "pcd",
    "1.e_experiencia_profissional_prejudicada": "experiencia_prejudicada",
    "1.i.2_regiao_onde_mora": "regiao",
    "1.l_nivel_de_ensino": "ensino",
    "1.m_área_de_formação": "area_de_formacao",

    # Situação profissional
    "2.a_situação_de_trabalho": "situacao_atual",
    "2.b_setor": "setor",
    "2.c_numero_de_funcionarios": "numero_de_funcionarios",
    "2.d_atua_como_gestor": "gestor",
    "2.f_cargo_atual": "cargo_atual",
    "2.g_nivel": "nivel",
    "2.h_faixa_salarial": "faixa_salario",
    "2.i_tempo_de_experiencia_em_dados": "experiencia_com_dados",
    "2.j_tempo_de_experiencia_em_ti": "experiencia_com_TI_engenharia",
    "2.k_satisfeito_atualmente": "satisfeito",

    # Modelos de trabalho
    "2.r_modelo_de_trabalho_atual": "modelo_de_trabalho",
    "2.s_modelo_de_trabalho_ideal": "modelo_de_trabalho_ideal",
    "2.t_atitude_em_caso_de_retorno_presencial": "volta_ao_presencial",

    # Mudança de emprego
    "2.n_planos_de_mudar_de_emprego_6m": "mudar_de_emprego",

    # Motivos de insatisfação
    "2.o.7_Plano de carreira e oportunidades de crescimento": "oportunidade",
    "2.o.1_Remuneração/Salário": "salario_mercado",
    "('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')": "lider",
    "2.o.9_Qualidade dos gestores e líderes": "lider",
    "('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')": "area",
    "5.a_objetivo_na_area_de_dados": "area",
    "('P2_l_5 ', 'Gostaria de receber mais benefícios')": "beneficios",
    "2.l.2_Benefícios": "beneficios",
    "('P2_l_6 ', 'O clima de trabalho/ambiente não é bom')": "clima",
    "2.o.5_Ambiente e clima de trabalho": "clima",
    "('P2_l_7 ', 'Falta de maturidade analítica na empresa')": "maturidade",
    "2.o.8_Maturidade da empresa em termos de tecnologia e dados": "maturidade",
    "3.c.1_Pensar na visão de longo prazo de dados": "visao",

    # Tamanho da equipe
    "3.a_numero_de_pessoas_em_dados": "tamanho_equipe",

    # Funções e desafios dos gestores
    "3.c.2_Organização de treinamentos e iniciativas": "treinamento",
    "3.c.3_Atração, seleção e contratação": "rh",
    "3.c.4_Decisão sobre contratação de ferramentas": "ferramentas",
    "6.a.6_Desenvolvo/cuido da manutenção de repositórios de dados baseados em streaming de eventos como Data Lakes e Data Lakehouses.": "data_lake",
    "2.l.9_Relação com os gestores e líderes": "gestor_equipe",
    "3.c.7_gestor da equipe de Inteligência Artificial e Machine Learning": "gestor_ia",
    "3.c.8_Apesar de ser gestor ainda atuo na parte técnica": "parte_tecnica",
    "3.d.5_Gestão de projetos envolvendo áreas multidisciplinares": "gestor_etapa",
    "3.c.10_Gestão de produtos de dados": "gestor_produto",
    "3.c.11_Gestão de pessoas": "desenvolvimento",
    "3.d_desafios_como_gestor": "desafios_gestor"
    }
    df.rename(columns=colunas, inplace=True)

    cols = [
        "idade", "genero", "cor", "ensino", "modelo_de_trabalho",
        "numero_de_funcionarios", "tamanho_equipe",
        "faixa_salario", "experiencia_com_dados", "satisfeito"
    ]
    df = df[cols].dropna()

    mask = (df["genero"] == "Prefiro não informar") | \
           (df["cor"] == "Prefiro não informar") | \
           (df["ensino"] == "Prefiro não informar")
    df = df.loc[~mask]

    return df


def preparar_dados(df):
    X = df.drop("satisfeito", axis=1)
    y = df["satisfeito"].astype(int)
    return X, y


def treinar_modelo(X, y, n_splits=10, random_state=16):
    cat_features = X.columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
        ]
    )

    pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("smote", SMOTE(random_state=random_state)),
        ("model", LogisticRegression(max_iter=1000))
    ])

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for train_idx, _ in kfold.split(X, y):
        pipeline.fit(X.iloc[train_idx], y.iloc[train_idx])

    return pipeline


if __name__ == "__main__":
    caminho = "State_of_data_2025.csv"
    df = carregar_dados(caminho)
    X, y = preparar_dados(df)
    modelo = treinar_modelo(X, y)

    joblib.dump(modelo, "classificar_satisfacao25.pkl", compress=9)
